#### 분산 분석(ANOVA): 여러 집단의 평균 차이를 통계적으로 유의미한지 검정하는 방법으로 주로 3개 이상의 집단을 비교할 때 사용

#### 일원 분산 분석
- 3개 이상의 집단 간의 평균 차이를 검증
- 3개 집단을 나누는 요인(원인)이 1개라서 일원
- 만약 집단을 나누는 요인이 1개이고 집단의 수가 2개일 때는 t-검정을 적용

In [3]:
# 기본 가정
    # 독립성: 각 집단의 관측치는 모두 다른 집단의 관측치들과 독립적이다 > 기본 가정
    # 정규성: 각 집단에서의 관측치는 정규분포를 따른다 > shapiro
    # 등분산성: 모든 집단에서의 관측치는 동일한 분산을 가진다 > levene

In [4]:
# 귀무가설: 모든 집단의 평균은 동일하다.
# 대립가설: 집단의 평균에는 차이가 있다.(적어도 두 개의 그룹)

In [5]:
# scipy.stats.f_oneway 사용하기
    # from scipy.stats.f_oneway(sample1, sample2, sample3 ...)
    
# 4 종류의 비료를 사용한 식물 성장 실험 결과 비교 (유의수준 0.05)
import pandas as pd
df = pd.DataFrame({
    'A': [10.5, 11.3, 10.8, 9.6, 11.1, 10.2, 10.9, 11.4, 10.5, 10.3],
    'B': [11.9, 12.4, 12.1, 13.2, 12.5, 11.8, 12.2, 12.9, 12.4, 12.3],
    'C': [11.2, 11.7, 11.6, 10.9, 11.3, 11.1, 10.8, 11.5, 11.4, 11.0],
    'D': [9.8, 9.4, 9.1, 9.5, 9.6, 9.9, 9.2, 9.7, 9.3, 9.4]
})

# 1 | 정규분포 확인
from scipy.stats import shapiro
print(shapiro(df['A']))
print(shapiro(df['B']))
print(shapiro(df['C']))
print(shapiro(df['D']))
# 모두 0.05 보다 크므로 귀무가설(정규분포를 따른다)를 채택

# 2 | 등분산성 확인
from scipy.stats import levene
print(levene(df['A'], df['B'], df['C'], df['D']))
# 0.05보다 크므로 귀무가설 채택(네 개의 그룹은 분산이 같음)

# 3 | 일원 분산 분석 진행
from scipy.stats import f_oneway
f_oneway(df['A'], df['B'], df['C'], df['D'])
# 0.05 보다 작으므로 귀무가설 기각, 대립가설 채택

ShapiroResult(statistic=0.9649054066073813, pvalue=0.8400161543468654)
ShapiroResult(statistic=0.9468040874196029, pvalue=0.6308700692815115)
ShapiroResult(statistic=0.9701646110856055, pvalue=0.892367306190296)
ShapiroResult(statistic=0.9752339025839644, pvalue=0.9346854448707653)
LeveneResult(statistic=1.9355354288758708, pvalue=0.14127835331346628)


F_onewayResult(statistic=89.12613851177174, pvalue=1.0018381522523723e-16)

In [6]:
# statsmodels를 이용해서 진행하기
# 판다스롤 데이터 불러와서 적용
df = pd.read_csv('fertilizer.csv')
display(df.head())
display(df.tail())

,비료,성장
0,A,10.5
1,A,11.3
2,A,10.8
3,A,9.6
4,A,11.1


,비료,성장
35,D,9.9
36,D,9.2
37,D,9.7
38,D,9.3
39,D,9.4


In [9]:
# '성장'에 대한 '비료' 요인의 영향을 분석하기 위해
    # Ordinary Least Squares 모델을 사용하고
    # anova linear model에 입력
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# ols에 입력 할 때는
    # '성장'에 대한 '비료'의 요인을 분석 > 성장 ~비교
    # 비료를 범주형으로 처리해서 넣기 > ~C(비료)
    # 로 컬럼명을 지정
model = ols("성장 ~C(비료)", df).fit()

# model로 anova lm에 넣기
print(anova_lm(model))
    # PR이 0.05보다 매우 작으므로
    # 비료가 성장에 영향을 안 준다는 귀무가설을 기각

            df    sum_sq    mean_sq          F        PR(>F)
C(비료)      3.0  43.21875  14.406250  89.126139  1.001838e-16
Residual  36.0   5.81900   0.161639        NaN           NaN


#### 이원 분산 분석
- 2개 이상의 독립적인 요인(factor)에 대해 각 요인이 종속 변수에 미치는 영향을 검증
- 각 요인에 대해 두 가지 이상의 수준이 있을 때 사용하며, 두 요인의 상호작용도 고려할 수 있음
- 예를 들어, 비료 종류와 온도라는 두 요인이 성장에 미치는 영향을 분석할 수 있음

In [10]:
# 기본 가정
    # 독립성: 각 그룹(요인 수준)의 관측치는 다른 그룹의 관측치들과 독립적이어야 한다.
    # 정규성: 각 그룹에서의 관측치는 정규분포를 따라야 한다. > shapiro
    # 등분산성: 모든 그룹에서의 관측치는 동일한 분산을 가져야 한다. > levene
    # 상호작용: 두 요인 간의 상호작용이 있는지를 확인할 수 있다.

In [ ]:
# 